# Ollama: Modelos de Lenguaje en Tu Computadora

## Objetivos de Aprendizaje

En este cuaderno vas a aprender:
- Qué es Ollama y por qué es útil ejecutar modelos localmente
- Cómo configurar y usar modelos de lenguaje sin APIs pagas
- Técnicas de prompting con system roles
- Extracción de datos estructurados (JSON) desde texto
- Procesamiento automatizado de archivos
- Creación de interfaces de usuario con Gradio

## ¿Qué es Ollama?

**Ollama** es una herramienta que te permite ejecutar modelos de lenguaje grandes (LLMs) en tu propia computadora, sin necesidad de conexión a internet ni APIs pagas.

### Ventajas de usar Ollama

1. **Privacidad**: Tus datos no salen de tu computadora
2. **Costo $0**: No pagás por tokens ni llamadas a APIs
3. **Sin límites**: Podés hacer todas las consultas que quieras
4. **Offline**: Funciona sin conexión a internet (después de descargar el modelo)
5. **Rápido**: Respuestas inmediatas si tenés buena GPU

### Casos de uso ideales

- Procesamiento de datos sensibles (historiales médicos, información personal)
- Prototipos y experimentos con muchas iteraciones
- Aplicaciones que requieren procesamiento en tiempo real
- Entornos sin acceso a internet
- Educación y aprendizaje

## Prerequisitos

Para usar este cuaderno necesitás:
1. **Ollama instalado** en tu sistema: https://ollama.ai/download
2. **Un modelo descargado**. En este cuaderno usamos Gemma 3
3. **Requisitos de hardware**:
   - Mínimo: 8GB RAM, CPU moderna
   - Recomendado: 16GB RAM + GPU NVIDIA

### Instalación rápida de Ollama

**En Linux/Mac**:
```bash
curl https://ollama.ai/install.sh | sh
```

**En Windows**:
Descargá el instalador desde https://ollama.ai/download

**Descargar modelo**:
```bash
# Modelo pequeño y rápido (recomendado para empezar)
ollama pull gemma:2b

# Modelo más potente (requiere más RAM)
ollama pull gemma:7b
```

## Configuración del Entorno

Instalamos las dependencias necesarias.

In [ ]:
# Instalación de librerías
!pip install -q ollama pandas gradio

In [ ]:
# Importaciones
import json
import os
import pandas as pd
from ollama import chat

# Configuración del modelo
# Si descargaste otro modelo, cambiá este nombre
MODEL_NAME = "gemma:2b"  # Modelo pequeño y rápido
# MODEL_NAME = "gemma:7b"  # Modelo más potente
# MODEL_NAME = "llama2"  # Alternativa popular

print(f"Configuración completada")
print(f"Modelo a usar: {MODEL_NAME}")
print("\nVerificá que Ollama esté corriendo: ollama list")

## Primera Consulta: Análisis de Sentimiento

Empecemos con algo simple: analizar el sentimiento de un comentario.

In [ ]:
# Comentario a analizar
comentario = """
Fui al cine a ver la nueva de Marvel y la verdad que una decepción total.
La trama no tiene sentido, los efectos especiales están bien pero ya aburren.
Encima me cobraron una fortuna por la entrada y el pochoclo.
No la recomiendo para nada.
"""

# Estructura del mensaje
mensaje = [
    {
        "role": "user",
        "content": f"Analiza el sentimiento de este comentario: {comentario}"
    }
]

print("Enviando consulta al modelo local...")
print(f"Comentario: {comentario}")

# Llamada a Ollama
response = chat(
    model=MODEL_NAME,
    messages=mensaje
)

# Resultado
respuesta = response.message.content
print("\n" + "="*80)
print("ANÁLISIS DEL MODELO:")
print("="*80)
print(respuesta)

## System Role: Definiendo el Comportamiento

El **system role** es un mensaje especial que define cómo debe comportarse el asistente. Es como darle instrucciones permanentes sobre:
- Su personalidad y tono
- Su expertise
- Reglas que debe seguir
- Formato de respuestas

### Comparación: Sin System Role vs Con System Role

In [ ]:
consulta = "¿Qué opinás de la última película de Tarantino?"

# SIN SYSTEM ROLE
print("="*80)
print("1. SIN SYSTEM ROLE (comportamiento genérico)")
print("="*80)

respuesta_generica = chat(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": consulta}]
)
print(respuesta_generica.message.content)

print("\n" + "="*80)
print("2. CON SYSTEM ROLE (crítico de cine especializado)")
print("="*80)

# CON SYSTEM ROLE
system_prompt_cine = """
Sos un crítico de cine argentino especializado en cine independiente y de autor.
Tenés 15 años de experiencia escribiendo para revistas culturales.
Tu estilo es:
- Análisis profundo de narrativa y dirección
- Referencias a cine clásico y contemporáneo
- Tono profesional pero accesible
- Comparaciones con otras obras del mismo director
Respondés en español rioplatense.
"""

respuesta_especializada = chat(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": system_prompt_cine},
        {"role": "user", "content": consulta}
    ]
)
print(respuesta_especializada.message.content)

## Extracción de Datos Estructurados (JSON)

Una de las aplicaciones más útiles de los LLMs es convertir texto no estructurado en datos estructurados (JSON).

### Caso de uso: Analizar reviews de productos

In [ ]:
def extract_json_from_text(text: str):
    """
    Extrae JSON de texto que puede contener explicaciones adicionales.
    Los modelos a veces agregan texto antes/después del JSON.
    """
    start = text.find('{')
    end = text.rfind('}')
    if start != -1 and end != -1 and end > start:
        candidate = text[start:end+1]
        try:
            return json.loads(candidate)
        except:
            return None
    return None

In [ ]:
# Review de un producto
review_producto = """
REVIEW: Auriculares Bluetooth XYZ-500
Comprador: Juan P. - Buenos Aires
Fecha: 15/03/2024

Los compré hace un mes y la verdad que re contentos. El sonido es excelente,
los graves se sienten mucho. La batería dura fácil 8 horas, perfecto para el laburo.
Lo único malo es que son medio pesados después de usarlos mucho rato, me cansan las orejas.
El precio me pareció razonable, 35 lucas, considerando la calidad que tienen.
La conexión Bluetooth es instantánea y nunca se corta.
Recomendación: 8/10, muy buenos para el precio.
"""

# System prompt para análisis estructurado
system_prompt_analisis = """
Sos un sistema de análisis de reviews que extrae información estructurada.
Tu tarea es convertir reviews en formato JSON con los siguientes campos:
- producto: nombre del producto
- comprador: nombre del comprador
- fecha: fecha de la review
- calificacion: número del 1 al 10
- aspectos_positivos: lista de aspectos positivos mencionados
- aspectos_negativos: lista de aspectos negativos mencionados
- precio_mencionado: precio si lo menciona, null si no
- sentimiento_general: "positivo", "negativo" o "neutral"

IMPORTANTE: Devolvé SOLO el JSON, sin explicaciones adicionales.
"""

# Prompt de extracción
prompt_extraccion = f"""
Extrae la información de esta review en formato JSON:

{review_producto}

Devolvé SOLO el JSON válido.
"""

# Procesamiento
print("Extrayendo datos estructurados de la review...")

response = chat(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": system_prompt_analisis},
        {"role": "user", "content": prompt_extraccion}
    ]
)

texto_respuesta = response.message.content
print("\nRespuesta del modelo:")
print(texto_respuesta)

# Extraer JSON
datos_estructurados = extract_json_from_text(texto_respuesta)

if datos_estructurados:
    print("\n" + "="*60)
    print("DATOS ESTRUCTURADOS EXTRAÍDOS:")
    print("="*60)
    print(json.dumps(datos_estructurados, indent=2, ensure_ascii=False))
    
    # Convertir a DataFrame
    df = pd.DataFrame([datos_estructurados])
    print("\nDataFrame generado:")
    print(df)
else:
    print("\nNo se pudo extraer JSON válido de la respuesta.")

## Procesamiento de Archivos: De Texto a Datos Estructurados

Una aplicación práctica: procesar archivos de texto y convertirlos en CSVs estructurados.

### Ejemplo: Análisis de comentarios de redes sociales

In [ ]:
# Creamos un archivo de ejemplo con comentarios
comentarios_ejemplo = """
COMENTARIOS DEL VIDEO: "RECORRIDA POR PALERMO SOHO 2024"
Canal: Buenos Aires Travel
Fecha: 10/03/2024
Reproducciones: 45,230

---
Usuario: @MartinBA
Excelente video! Me encanta Palermo, tiene los mejores bares y restaurantes de la ciudad.
La zona está re linda ahora, mucho mejor que hace unos años.

Usuario: @SofiTraveler
Muy bueno el recorrido, pero la verdad que Palermo está carísimo.
Un café sale una fortuna. Prefiero San Telmo que tiene más onda y es más barato.

Usuario: @LucasARG
¡Qué nostalgia! Viví en Palermo 5 años y extraño un montón.
Los domingos en la plaza Serrano eran lo mejor. Gracias por el video.

Usuario: @TuristaChile
Voy en mayo a Buenos Aires, este video me sirve un montón para planificar.
¿Alguien sabe si los bares de la zona tienen wifi? Necesito trabajar remoto.

Usuario: @PalermoLocal
Vivo acá hace 10 años. El barrio cambió mucho, se llenó de turistas.
Ya no es lo que era, perdió un poco de autenticidad. Pero igual es lindo.
"""

# Guardamos el archivo
archivo_comentarios = "comentarios_palermo.txt"
with open(archivo_comentarios, "w", encoding="utf-8") as f:
    f.write(comentarios_ejemplo)

print(f"Archivo de ejemplo creado: {archivo_comentarios}")

In [ ]:
# System prompt para análisis de comentarios
system_prompt_comentarios = """
Sos un analista de redes sociales que procesa comentarios de YouTube.
Extraes información estructurada de cada comentario:
- Usuario
- Sentimiento (positivo/negativo/neutral/pregunta)
- Tema principal
- Menciona precio (sí/no)
- Intención (opinión/pregunta/nostalgia/recomendación)

Generás un CSV con columnas: usuario,sentimiento,tema,menciona_precio,intencion,texto_resumen
"""

# Leer el archivo
with open(archivo_comentarios, "r", encoding="utf-8") as f:
    contenido = f.read()

# Prompt para generar CSV
prompt_csv = f"""
Analiza estos comentarios y genera un CSV con la estructura:
usuario,sentimiento,tema,menciona_precio,intencion,texto_resumen

COMENTARIOS:
{contenido}

Devolvé SOLO el CSV, sin explicaciones. Primera fila: nombres de columnas.
"""

print("Procesando comentarios...")

response = chat(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": system_prompt_comentarios},
        {"role": "user", "content": prompt_csv}
    ]
)

csv_resultado = response.message.content

print("\nCSV generado:")
print("="*80)
print(csv_resultado)

# Intentar cargar como DataFrame
try:
    from io import StringIO
    df_comentarios = pd.read_csv(StringIO(csv_resultado))
    
    print("\n" + "="*80)
    print("ANÁLISIS DE COMENTARIOS:")
    print("="*80)
    print(df_comentarios)
    
    # Estadísticas
    print("\nESTADÍSTICAS:")
    print(f"Total de comentarios: {len(df_comentarios)}")
    if 'sentimiento' in df_comentarios.columns:
        print("\nDistribución de sentimientos:")
        print(df_comentarios['sentimiento'].value_counts())
    
    # Guardar CSV
    df_comentarios.to_csv('analisis_comentarios.csv', index=False)
    print("\nDatos guardados en: analisis_comentarios.csv")
    
except Exception as e:
    print(f"\nNo se pudo parsear como CSV: {e}")
    print("El modelo quizás agregó texto extra antes/después del CSV")

## Interfaz de Usuario con Gradio

Crear una interfaz web simple para que usuarios no técnicos puedan usar el modelo.

### Aplicación: Analizador de Sentimiento de Texto

In [ ]:
import gradio as gr

def analizar_sentimiento(texto, detalle):
    """
    Analiza el sentimiento de un texto usando el modelo local.
    
    Parámetros:
    - texto: El texto a analizar
    - detalle: Nivel de detalle del análisis
    """
    if not texto.strip():
        return "Por favor ingresá un texto para analizar."
    
    # Configurar el prompt según el nivel de detalle
    if detalle == "Simple":
        prompt = f"""Analiza el sentimiento de este texto y respondé en una palabra:
        Positivo, Negativo o Neutral.
        
        Texto: {texto}"""
    else:
        prompt = f"""Analiza detalladamente el sentimiento de este texto.
        Incluye:
        - Sentimiento general (positivo/negativo/neutral)
        - Intensidad (baja/media/alta)
        - Emociones detectadas
        - Tono del texto
        
        Texto: {texto}"""
    
    try:
        response = chat(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.message.content
    except Exception as e:
        return f"Error: {str(e)}\n\nVerificá que Ollama esté corriendo."

# Crear la interfaz
with gr.Blocks(title="Analizador de Sentimiento Local") as demo:
    gr.Markdown("""
    # Analizador de Sentimiento con IA Local
    ### Procesamiento 100% privado en tu computadora
    
    Esta herramienta usa Ollama para analizar el sentimiento de cualquier texto
    sin enviar datos a servidores externos.
    """)
    
    with gr.Row():
        with gr.Column():
            texto_input = gr.Textbox(
                label="Texto a analizar",
                placeholder="Pegá acá el texto que querés analizar...",
                lines=5
            )
            
            detalle_input = gr.Radio(
                choices=["Simple", "Detallado"],
                value="Simple",
                label="Nivel de detalle"
            )
            
            btn_analizar = gr.Button(
                "Analizar Sentimiento",
                variant="primary"
            )
        
        with gr.Column():
            resultado_output = gr.Textbox(
                label="Resultado del análisis",
                lines=10
            )
    
    # Ejemplos predefinidos
    gr.Examples(
        examples=[
            ["La película estuvo increíble, la mejor que vi este año!", "Simple"],
            ["No me gustó para nada, perdí mi tiempo y mi plata.", "Detallado"],
            ["Estuvo bien, nada del otro mundo pero entretenida.", "Detallado"],
        ],
        inputs=[texto_input, detalle_input]
    )
    
    # Conectar botón con función
    btn_analizar.click(
        fn=analizar_sentimiento,
        inputs=[texto_input, detalle_input],
        outputs=resultado_output
    )
    
    gr.Markdown("""
    ---
    **Características de Privacidad:**
    - Todo el procesamiento se hace localmente
    - Ningún dato se envía a servidores externos
    - Funciona sin conexión a internet
    - Costo $0 en APIs
    """)

# Lanzar la interfaz
print("Iniciando interfaz web...")
print(f"Modelo: {MODEL_NAME}")
print("La interfaz se abrirá en tu navegador")

demo.launch(
    server_name="127.0.0.1",
    server_port=7860,
    share=False  # No compartir públicamente
)

## Resumen y Conceptos Clave

### Lo que aprendimos:

1. **Ollama**: Plataforma para ejecutar LLMs localmente
   - Privacidad total
   - Sin costos de API
   - Funciona offline

2. **System Role**: Define el comportamiento del asistente
   - Personalidad y tono
   - Reglas y restricciones
   - Formato de respuestas

3. **Extracción de JSON**: Convertir texto no estructurado en datos
   - Siempre pedir "SOLO JSON"
   - Usar función de extracción robusta
   - Validar resultados

4. **Procesamiento de archivos**: Automatización de análisis
   - Leer archivos de texto
   - Generar CSVs estructurados
   - Crear DataFrames para análisis

5. **Interfaces con Gradio**: Democratizar el acceso
   - UI simple para usuarios no técnicos
   - Ejemplos predefinidos
   - Validación de inputs

### Comparación: Ollama vs APIs Cloud

| Característica | Ollama Local | APIs Cloud (OpenAI/Gemini) |
|----------------|--------------|----------------------------|
| Privacidad | Total | Datos en servidores externos |
| Costo | $0 (después de HW) | Por token/request |
| Velocidad | Depende de tu HW | Generalmente rápido |
| Calidad | Modelos más pequeños | Modelos muy potentes |
| Internet | No necesario | Obligatorio |
| Setup | Requiere instalación | Instant

o con API key |

### Cuándo usar cada opción:

**Usar Ollama cuando**:
- Necesitás privacidad absoluta
- Hacés muchas consultas (costo)
- Trabajás con datos sensibles
- No tenés conexión a internet confiable
- Querés aprender sin gastar

**Usar APIs Cloud cuando**:
- Necesitás la mejor calidad posible
- No tenés hardware potente
- Querés empezar rápido sin configuración
- El volumen de consultas es bajo

### Próximos pasos:

Ahora que conocés tanto APIs cloud como ejecución local, podés:
- Combinar ambas estrategias (prototipos con Ollama, producción con cloud)
- Implementar sistemas RAG con embeddings locales
- Crear aplicaciones que protegen la privacidad del usuario
- Experimentar sin límites de costo

## Glosario

**Ollama**: Plataforma open-source para ejecutar modelos de lenguaje localmente en tu computadora.

**System Role**: Mensaje especial que define el comportamiento global del asistente de IA.

**Local LLM**: Modelo de lenguaje grande que se ejecuta en tu propia computadora, sin necesidad de internet.

**Inferencia**: Proceso de usar un modelo entrenado para generar respuestas (diferente de entrenamiento).

**Gradio**: Librería de Python para crear interfaces web simples para modelos de ML.

**GGUF**: Formato de archivo optimizado para almacenar y ejecutar LLMs de forma eficiente.

**Quantization**: Técnica para reducir el tamaño de modelos (ej: 4-bit) manteniendo calidad aceptable.

**Context Window**: Cantidad máxima de tokens (texto) que el modelo puede procesar de una vez.

**Prompt Engineering**: Arte de escribir instrucciones efectivas para obtener mejores respuestas de LLMs.

## Preguntas Frecuentes

**P: ¿Cuánto espacio en disco ocupan los modelos?**

R: Depende del tamaño. Gemma 2B ocupa ~1.4GB, Gemma 7B ~4.8GB, Llama2 7B ~3.8GB. Los modelos más grandes (70B+) pueden ocupar 40GB o más.

---

**P: ¿Necesito una GPU para usar Ollama?**

R: No es obligatorio, pero MUY recomendado. Con GPU (NVIDIA) las respuestas son casi instantáneas. Con solo CPU funciona pero puede tardar varios segundos por respuesta.

---

**P: ¿Puedo usar modelos diferentes para diferentes tareas?**

R: Sí. Podés tener varios modelos instalados y cambiar según la tarea. Modelos pequeños para análisis rápidos, modelos grandes para tareas complejas.

---

**P: ¿Cómo actualizo un modelo?**

R: Simplemente ejecutá `ollama pull nombre-modelo` de nuevo. Ollama solo descarga las diferencias si hay una versión nueva.

---

**P: ¿Puedo entrenar o fine-tunear modelos con Ollama?**

R: Ollama es solo para inferencia (usar modelos). Para entrenar o fine-tunear necesitás otras herramientas (Hugging Face Transformers, LLaMA.cpp, etc.).

---

**P: ¿Es legal usar estos modelos comercialmente?**

R: Depende de la licencia del modelo. Llama 2 tiene restricciones comerciales si tenés muchos usuarios. Gemma es más permisiva. Siempre revisá la licencia específica del modelo.

## Referencias y Recursos

**Ollama**:
- Sitio oficial: https://ollama.ai
- Documentación: https://github.com/ollama/ollama
- Librería modelos: https://ollama.ai/library

**Modelos Recomendados**:
- Gemma (Google): Buena calidad, open-source, varias versiones
- Llama 2 (Meta): Popular, bien documentado
- Mistral: Excelente calidad/tamaño, muy eficiente
- Phi-2 (Microsoft): Modelo pequeño y sorprendentemente capaz

**Gradio**:
- Documentación: https://gradio.app/docs
- Ejemplos: https://gradio.app/demos

**Comunidad**:
- r/LocalLLaMA (Reddit): Comunidad activa de LLMs locales
- Ollama Discord: Soporte y discusiones

**Papers Relevantes**:
- "Llama 2: Open Foundation and Fine-Tuned Chat Models"
- "Gemma: Open Models Based on Gemini Research and Technology"
- "GGML: Large Language Models for Everyone"